# Models' predictions evaluation

**NOTEBOOK GOAL**: Evaluate the predicted NumberOfSales values.


Compared predictions:

- **1_RFR** - Notebook 5.3 Random forrest
- **2_XGB** - Notebook 6.4 XGBoost
- **3_AVG** - Notebook 7.0 AVG Monthly average
- **4_ENS_AVG** - Notebook 8.0 Ensamble by averaging all the previous models

In [1]:
from import_man import *
import collections

from BIP import get_BIP_error, apply_BIP_submission_format

### Load predicted tests

**NOTE** If you cannot load the followig datasets, please go to the corresponding notebook and run it to generate the related dataset file. 

In [2]:
dfs_dict = collections.OrderedDict()
# the following dataset will be evaluated

In [3]:
dfs_dict['RFR'] = pd.read_csv('./dataset/test_m12_53_RFR_on_prep.csv')

In [4]:
dfs_dict['XGB'] = pd.read_csv('./dataset/test_m12_64_Model_XGBoost_final.csv')

In [5]:
dfs_dict['AVG'] = pd.read_csv('./dataset/test_m12_70_Model_monthly_average.csv')

In [6]:
# apply the apply_BIP_submission_format to all the dataframes
for mdl_lbl, df in dfs_dict.items():
    dfs_dict[mdl_lbl] = apply_BIP_submission_format(df)

In [7]:
# The following dataset is already in the BIP submission format
dfs_dict['ENS_AVG'] = pd.read_csv('./dataset/test_m12_82_Ensemble_average.csv')

## Evaluation metrics

1. **[EVS] Explained Variance Score**  [(reference)](http://scikit-learn.org/stable/modules/model_evaluation.html#explained-variance-score)

    The mean_absolute_error function computes mean absolute error, a risk metric corresponding to the expected value of the absolute error loss or l1-norm loss.
    
    $$ \texttt{explained__variance}(y, \hat{y}) = 1 - \frac{Var\{ y - \hat{y}\}}{Var\{y\}} $$
    
    The best possible score is 1.0, lower values are worse.
    
2. **[MAE] Mean absolute error**  [(reference)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-absolute-error)

    The mean_squared_error function computes mean square error, a risk metric corresponding to the expected value of the squared (quadratic) error or loss.
   
    $$ \text{MAE}(y, \hat{y}) = \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}}-1} \left| y_i - \hat{y}_i \right| $$
    
3. **[MSE] Mean squared error**  [(reference)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-error)

    The mean_squared_error function computes mean square error, a risk metric corresponding to the expected value of the squared (quadratic) error or loss.
    
    $$ \text{MSE}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2 $$
    
4. **[RMSE] Root mean squared error**

    $$ \text{RMSE}(y, \hat{y}) = \sqrt{\frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (y_i - \hat{y}_i)^2} $$
    
5. **[MSLE] Mean squared logarithmic error**  [(reference)](http://scikit-learn.org/stable/modules/model_evaluation.html#mean-squared-logarithmic-error)
    
    The mean_squared_log_error function computes a risk metric corresponding to the expected value of the squared logarithmic (quadratic) error or loss.
    
    $$ \text{MSLE}(y, \hat{y}) = \frac{1}{n_\text{samples}} \sum_{i=0}^{n_\text{samples} - 1} (\log_e (1 + y_i) - \log_e (1 + \hat{y}_i) )^2 $$
    
    Where log_e (x) means the natural logarithm of x. This metric is best to use when targets having exponential growth, such as population counts, average sales of a commodity over a span of years etc. Note that this metric penalizes an under-predicted estimate greater than an over-predicted estimate.

6. **[MedAE] Median absolute error**  [(reference)](http://scikit-learn.org/stable/modules/model_evaluation.html#median-absolute-error)

    The median_absolute_error is particularly interesting because it is robust to outliers. The loss is calculated by taking the median of all absolute differences between the target and the prediction.
    
    $$ \text{MedAE}(y, \hat{y}) = \text{median}(\mid y_1 - \hat{y}_1 \mid, \ldots, \mid y_n - \hat{y}_n \mid)$$
    
7. **[R²] R²score, the coefficient of determination**  [(reference)](http://scikit-learn.org/stable/modules/model_evaluation.html#r2-score-the-coefficient-of-determination)

    The r2_score function computes R², the coefficient of determination. It provides a measure of how well future samples are likely to be predicted by the model. Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). A constant model that always predicts the expected value of y, disregarding the input features, would get a R^2 score of 0.0.
    
    $$ R^2(y, \hat{y}) = 1 - \frac{\sum_{i=0}^{n_{\text{samples}} - 1} (y_i - \hat{y}_i)^2}{\sum_{i=0}^{n_\text{samples} - 1} (y_i - \bar{y})^2}$$
    
    where
    
    $$ \bar{y} =  \frac{1}{n_{\text{samples}}} \sum_{i=0}^{n_{\text{samples}} - 1} y_i $$

In [8]:
list(dfs_dict.values())[0].head()

,StoreID,Month,Target,NumberOfSales
0,1000,1,182917,201451.830901
1,1000,2,166161,184721.490504
2,1001,1,95745,83185.502132
3,1001,2,88423,72102.648063
4,1002,1,121995,134334.706754


In [9]:
from sklearn import metrics

from math import sqrt

def print_regr_stats(df):
    y_true = df['Target']
    y_pred = df['NumberOfSales']
    
    stats = collections.OrderedDict()
    stats['EVS']   = metrics.explained_variance_score(y_true, y_pred)
    stats['MAE']   = metrics.mean_absolute_error(y_true, y_pred)
    stats['MSE']   = metrics.mean_squared_error(y_true, y_pred)
    stats['RMSE']  = sqrt(stats['MSE'])
    stats['MSLE']  = metrics.mean_squared_log_error(y_true, y_pred)
    stats['MedAE'] = metrics.median_absolute_error(y_true, y_pred)
    stats['R2']    = metrics.r2_score(y_true, y_pred)
    
    print("Explained Variance Score               [EVS] ", stats['EVS'])
    print("Mean absolute error                    [MAE] ", stats['MAE'])
    print("Mean squared error                     [MSE] ", stats['MSE'])
    print("Root mean squared error               [RMSE] ", stats['RMSE'])
    print("Mean squared logarithmic error        [MSLE] ", stats['MSLE'])
    print("Median absolute error (reference)    [MedAE] ", stats['MedAE'])
    print("R²score, coefficient of determination   [R²] ", stats['R2'])
    
    return stats


In [10]:
models_stats = collections.OrderedDict([(k, []) for k in ['EVS','MAE', 'MSE', 'RMSE', 'MSLE', 'MedAE', 'R2']])


for mdl_lbl, df in dfs_dict.items():
    print('................................ ' + mdl_lbl + '................................')
    stats = print_regr_stats(df)
    
    # add the model statistics to the main index
    for name, val in stats.items():
        models_stats[name].append((mdl_lbl, val))
        
    print('\n')
    
    
print("\n\nSTATISTICS COMPARISON\n")

for name, l in models_stats.items():
    print(name)
    pprint(list(enumerate(sorted(l, key=lambda tup: tup[1]))))


................................ RFR................................
Explained Variance Score               [EVS]  0.9516157191345931
Mean absolute error                    [MAE]  8403.387817765266
Mean squared error                     [MSE]  130439386.0449889
Root mean squared error               [RMSE]  11421.006349923324
Mean squared logarithmic error        [MSLE]  0.00879432389306739
Median absolute error (reference)    [MedAE]  6596.779882267692
R²score, coefficient of determination   [R²]  0.9266278413635942


................................ XGB................................
Explained Variance Score               [EVS]  0.9554457225986508
Mean absolute error                    [MAE]  6066.938750547397
Mean squared error                     [MSE]  79212072.19523914
Root mean squared error               [RMSE]  8900.116414701502
Mean squared logarithmic error        [MSLE]  0.005160802533279368
Median absolute error (reference)    [MedAE]  4035.7363999999943
R²score, coefficie

## BIP error

In [11]:
for mdl_lbl, df in dfs_dict.items():
    print('................................ ' + mdl_lbl + '................................')
    get_BIP_error(df, already_BIP_format=True)
    print('\n')

................................ RFR................................
BIP total error: 0.06939371779958461


................................ XGB................................
BIP total error: 0.04913953326583034


................................ AVG................................
BIP total error: 0.05346477095060351


................................ ENS_AVG................................
BIP total error: 0.0454826943863305


